Setup

In [7]:
import pandas as pd

# Set data files paths
FLOWS_PATH = "./data/SQLI-BFXSS/29-03-2024/netflows.csv"
CPU_PATH = "./data/SQLI-BFXSS/29-03-2024/CPU Usage-data-as-joinbyfield-2024-03-29 18_20_54.csv"
save_directory = "./output/SQLI-BFXSS"
start_date = '2024-03-29 05:39:13'
end_date = '2024-03-29 14:51:30'

df_flow = pd.read_csv(FLOWS_PATH)
print(df_flow.shape)
df_flow.head()

(28416, 80)


,class,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
0,[0],58270,6,2024-03-29 05:35:10,30126.810074,1.394107e+04,199.158158,132.772105,66.386053,4,...,0.0,0.0,0.0,66.0,78.0,0,4,2,264,156
1,[0],45010,6,2024-03-29 05:35:11,30352.115631,1.601206e+04,230.626428,131.786530,98.839898,4,...,0.0,0.0,0.0,66.0,74.0,0,4,3,264,222
2,[0],58286,6,2024-03-29 05:35:11,30293.226242,1.604319e+04,231.074761,132.042720,99.032040,4,...,0.0,0.0,0.0,66.0,74.0,0,4,3,264,222
3,[0],10249,6,2024-03-29 05:35:13,78.678131,1.626882e+06,25420.024242,12710.012121,12710.012121,1,...,0.0,0.0,0.0,74.0,54.0,0,1,1,74,54
4,[0],2381,6,2024-03-29 05:35:14,76.293945,1.677722e+06,26214.400000,13107.200000,13107.200000,1,...,0.0,0.0,0.0,74.0,54.0,0,1,1,74,54


Time filtering

In [8]:
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

print(df_flow.shape)

# Filter rows between start_date and end_date
df_flow = df_flow[(df_flow['timestamp'] >= start_date) & (df_flow['timestamp'] <= end_date)]

print(df_flow.shape)

# Replace class values ([0] -> 0 and [1] -> 1)
df_flow['class'] = df_flow['class'].replace({'[0]': 0, '[1]': 1})
# filtered_df.head()
df_flow[df_flow['class']==1]

(28416, 80)
(17082, 80)


C:\Users\hr_mo\AppData\Local\Temp\ipykernel_20328\2052669154.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_flow['class'] = df_flow['class'].replace({'[0]': 0, '[1]': 1})


,class,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
690,1,22,6,2024-03-29 05:57:13,1.421854e+05,4409.733920,63.297616,35.165342,28.132274,5,...,0.0,0.0,0.0,62.800000,78.25,0,5,4,314,313
6648,1,22,6,2024-03-29 09:07:12,2.119062e+05,1826.279787,23.595346,14.157208,9.438138,3,...,0.0,0.0,0.0,68.666667,90.50,0,3,2,206,181
8948,1,22,6,2024-03-29 10:22:48,1.215508e+05,5158.337158,74.043117,41.135065,32.908052,5,...,0.0,0.0,0.0,62.800000,78.25,0,5,4,314,313
9203,1,22,6,2024-03-29 10:30:58,8.178034e+06,209.463550,1.222788,0.733673,0.489115,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
9479,1,22,6,2024-03-29 10:39:27,7.353058e+06,232.964311,1.359978,0.815987,0.543991,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
9580,1,22,6,2024-03-29 10:42:17,7.016068e+06,244.153832,1.425300,0.855180,0.570120,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
9855,1,22,6,2024-03-29 10:50:59,6.494335e+06,263.768329,1.539803,0.923882,0.615921,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
9957,1,22,6,2024-03-29 10:53:48,5.709574e+06,300.022384,1.751444,1.050866,0.700578,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
10041,1,22,6,2024-03-29 10:56:44,5.981776e+06,286.369789,1.671744,1.003047,0.668698,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289
10129,1,22,6,2024-03-29 10:59:27,6.033285e+06,283.924920,1.657472,0.994483,0.662989,6,...,0.0,0.0,0.0,70.666667,322.25,0,6,4,424,1289


In [9]:
# Save flow data
df_flow.to_csv(save_directory + "flows.csv", index=False)

Column filtering

In [10]:
# df_flow = df_flow.get(["timestamp", "class", "flow_duration", "flow_byts_s", "flow_pkts_s", "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts"])
# print(df_flow.shape)
# df_flow.head()

Prepare CPU data

In [11]:
df_cpu = pd.read_csv(CPU_PATH)
df_cpu = df_cpu.rename(columns={'Time': 'timestamp'})
df_cpu = df_cpu.drop("Value 1", axis=1)
# Merge CPU values by sum
df_cpu["cpu_total"] = df_cpu.drop("timestamp", axis=1).sum(axis=1)
df_cpu.head()

,timestamp,classifier,controller,database,sniffer,cpu_total
0,2024-03-29 05:39:10,0.0286,0.00574,0.000469,0.843,0.877809
1,2024-03-29 05:39:15,0.0286,0.00574,0.000469,0.843,0.877809
2,2024-03-29 05:39:20,0.0286,0.00574,0.000469,0.843,0.877809
3,2024-03-29 05:39:25,0.0286,0.00574,0.000469,0.843,0.877809
4,2024-03-29 05:39:30,0.0142,0.00483,0.000972,0.039,0.059002


In [12]:
# Save CPU data
df_cpu.to_csv(save_directory + "cpu.csv", index=False)

Convert timestamp columns to datetime objects

In [13]:
df_cpu['timestamp'] = pd.to_datetime(df_cpu['timestamp'])
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

Data mapping (method 1)

In [14]:
df_flow1 = df_flow

# Create a new column in the network dataframe to store associated cpu records
df_flow1['cpu'] = None

# Loop over each flow row
for index, flow_row in df_flow1.iterrows():
    # Find the nearest cpu record according to time
    nearest_cpu_record = df_cpu.iloc[(df_cpu['timestamp'] - flow_row['timestamp']).abs().argsort()[0]]

    # Assign the nearest cpu record to the current flow row
    df_flow1.at[index, 'cpu'] = nearest_cpu_record['cpu_total']

# Convert all timestamps to total seconds from the first timestamp
df_flow1 = df_flow1.sort_values(by='timestamp')
first_timestamp = df_flow1['timestamp'].iloc[0]
df_flow1['timestamp'] = (df_flow1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_flow1.to_csv(save_directory + "data1.csv", index=False)

# Print the updated network dataframe with associated cpu records
print(df_flow1)

       class  dst_port  protocol  timestamp  flow_duration   flow_byts_s  \
105        0     10249         6        0.0   6.842613e+01  1.870630e+06   
106        0      2381         6        1.0   7.104874e+01  1.801580e+06   
109        0     31808         6        1.0   1.272177e+06  7.766215e+02   
113        0     10259         6        6.0   2.598763e+01  4.925421e+06   
114        0       500        17        7.0   0.000000e+00  0.000000e+00   
...      ...       ...       ...        ...            ...           ...   
17202      0     10249         6    33120.0   3.647804e+01  3.508960e+06   
17203      0      2381         6    33121.0   3.004074e+01  4.260880e+06   
17204      0     10259         6    33126.0   2.765656e+01  4.628198e+06   
17236      0      6443         6    33127.0   9.124560e+07  3.112479e+01   
17208      0       500        17    33137.0   0.000000e+00  0.000000e+00   

        flow_pkts_s    fwd_pkts_s    bwd_pkts_s  tot_fwd_pkts  ...  \
105    29228.5993

Data mapping (method 2)

In [15]:
df_cpu1 = df_cpu

# Function to aggregate flows for each CPU record
def aggregate_flows(cpu_row, network_df):
    # Get the current and next CPU timestamps
    current_timestamp = cpu_row['timestamp']
    next_timestamp = df_cpu1.loc[cpu_row.name + 1, 'timestamp'] if cpu_row.name + 1 < len(df_cpu) else pd.Timestamp.max

    # Filter flows that fall within the time interval
    flows_in_interval = network_df[(network_df['timestamp'] >= current_timestamp) & (network_df['timestamp'] < next_timestamp)]

    # Aggregate flow values
    aggregated_values = flows_in_interval.drop(columns=['timestamp']).sum()

     # Add a column for the number of aggregated flows
    aggregated_values['num_flows'] = len(flows_in_interval)

    return aggregated_values

# Loop over each CPU row
for index, cpu_row in df_cpu1.iterrows():
    # Aggregate flows for the current CPU row
    aggregated_values = aggregate_flows(cpu_row, df_flow)

    # Add aggregated flow values to the CPU row
    df_cpu1.loc[index, aggregated_values.index] = aggregated_values.values

# Fill NaN values with 0
df_cpu1.fillna(0, inplace=True)

# Convert all timestamps to total seconds from the first timestamp
df_cpu1 = df_cpu1.sort_values(by='timestamp')
first_timestamp = df_cpu1['timestamp'].iloc[0]
df_cpu1['timestamp'] = (df_cpu1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_cpu1.to_csv(save_directory + "data2.csv", index=False)

# Print the updated CPU dataframe
print(df_cpu1)

      timestamp  classifier  controller  database  sniffer  cpu_total  class  \
0           0.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
1           5.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
2          10.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
3          15.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
4          20.0     0.01420     0.00483  0.000972  0.03900   0.059002    0.0   
...         ...         ...         ...       ...      ...        ...    ...   
6624    33120.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6625    33125.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6626    33130.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6627    33135.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6628    33140.0     0.01580     0.00215  0.000325  0.00901   0.027285    0.0   

      dst_port  protocol  flow_duration